<a href="https://colab.research.google.com/github/psriraj17/NLP-3/blob/main/semi_supervised_nlp_ssr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The entire code was extracted from "https://github.com/gregfromstl/word_segmentation/blob/master/semi-supervised.ipynb"

In [112]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [113]:
import torch
from math import log
import numpy as np
torch.set_printoptions(precision=10)

device = torch.device('cuda:2' if torch.cuda.is_available() else 'cpu')

In [150]:
train_file = '/content/drive/MyDrive/train_semi.tsv'
val_split = 0.95

states = {
    'B' :0,
    'I' :1,
}

In [115]:
def load_data(file):
  print("Loading data from file {}...".format(file))
  file = open(file, 'r')
  data = []
  for line in file:
      pieces = line.rstrip("\n").split("\t")
      data.append(pieces)
  print("Loaded {} sentences".format(len(data)))
  return data


In [116]:
train_data = list(load_data(train_file))

Loading data from file /content/drive/MyDrive/train_semi.tsv...
Loaded 12812 sentences


In [117]:
print("Splitting data...")
num_train_samples = int(len(train_data)*(1-val_split))
val_data = train_data[num_train_samples:]
print(len(val_data),'validation set')
train_data = train_data[:num_train_samples]
print(len(train_data),"training set")

Splitting data...
12172 validation set
640 training set


In [118]:
print(train_data[0])

['konsiltan', 'k-on-s-i-l-t-an']


In [119]:
def compute_probabilities_from_counts(counts_dict):
    counts_sum = sum(counts_dict.values())
    probabilities_dict = {}
    for count_id in counts_dict:
        count = counts_dict[count_id]
        probabilities_dict[count_id] = count / counts_sum
    assert round(sum(probabilities_dict.values()), 2) == 1.0, "All probabilities should sum to 1 but got {}".format(round(sum(probabilities_dict.values()), 2))
    return probabilities_dict

In [120]:
def key_with_max_val(d):
    """https://stackoverflow.com/questions/268272/getting-key-with-maximum-value-in-dictionary"""
    v = list(d.values())
    k = list(d.keys())
    return k[v.index(max(v))]

In [121]:
def generate_initial_state_probabilities(data):
    initial_state_counts = states.copy()
    # equal probability of starting
    for state in initial_state_counts:
        initial_state_counts[state] += 1
    initial_state_probabilities = compute_probabilities_from_counts(initial_state_counts)
    return initial_state_probabilities

In [122]:
def generate_transition_state_probabilities(data):
    # create a dictionary with two levels, the first being the previous state and the second being the current state
    transition_state_counts = {state: states.copy() for state in states}
    # since we enumerate over a list that excludes the first item, the enumeration index is one behind
    for prev_idx, word in enumerate(data[1:]):
        prev_state = data[prev_idx][1]
        current_state = word[1]
        if prev_state in transition_state_counts and current_state in transition_state_counts[prev_state]:
            transition_state_counts[prev_state][current_state] += 1
    # setting STOP count to 1 for all states to avoid zeros
    for state in transition_state_counts:
        transition_state_counts[state]['STOP'] = 1
    transition_state_probabilities = {state: {} for state in states}
    for prev_state in transition_state_counts:
        transition_state_probabilities[prev_state] = compute_probabilities_from_counts(transition_state_counts[prev_state])
    return transition_state_probabilities

In [123]:
def generate_emission_probabilities(data, all_observations):
    vocab = {obs: 1 for obs in set(all_observations)}
    emission_counts_by_state = {state: vocab for state in states}
    for word_state_pair in data:
        word, state = word_state_pair
        if state in emission_counts_by_state:
            # initialize word in state dict if the first occurrence of word X in state Y
            if word not in emission_counts_by_state[state]:
                emission_counts_by_state[state][word] = 0
            emission_counts_by_state[state][word] += 1
    emission_probabilities_by_state = {state: {} for state in states}
    for state in emission_counts_by_state:
        emission_probabilities_by_state[state] = compute_probabilities_from_counts(emission_counts_by_state[state])
    return emission_probabilities_by_state

In [124]:
def fit(data, vocab):
  print("Fitting model to provided dataset...")
  initial_state_probabilities = generate_initial_state_probabilities(data)
  transition_probabilities = generate_transition_state_probabilities(data)
  emission_probabilities = generate_emission_probabilities(data, vocab)
  print("Model ready.")
  return initial_state_probabilities, transition_probabilities, emission_probabilities

In [125]:
val_sequence = [pair[0] for pair in val_data]
val_labels = [pair[1] for pair in val_data]
initial, transition, emission  = fit(train_data, val_sequence)

Fitting model to provided dataset...
Model ready.


In [126]:
def get_forward_prob(observations):
    forward_probabilities = {}
    for idx, observation in enumerate(observations):
        for state in initial:
            if idx == 0:
                probability = initial[state]*emission[state][observation] if observation in emission[state] else 0
                forward_probabilities[state] = torch.DoubleTensor([probability])
            else:
                probability = 0
                for prev_state in transition:
                    probability += forward_probabilities[prev_state][idx-1]*transition[prev_state][state]
                probability *= emission[state][observation]
                forward_probabilities[state] = torch.cat((forward_probabilities[state], torch.DoubleTensor([probability])))
    return forward_probabilities

In [127]:
def get_backward_prob(observations):
    backward_probabilities = {}
    for i in range(len(observations)):
        idx = len(observations)-i-1
        for state in initial:
            if idx == len(observations)-1:
                probability = transition[state]['STOP']
                backward_probabilities[state] = torch.DoubleTensor([probability])
            else:
                probability = 0
                for next_state in transition:
                    probability += backward_probabilities[next_state][0]*transition[state][next_state]*emission[next_state][observations[idx+1]]
                backward_probabilities[state] = torch.cat((torch.DoubleTensor([probability]), backward_probabilities[state]))
    return backward_probabilities

In [128]:
def get_state_total_prob(forward_prob: dict, backward_prob: dict) -> dict:
    state_total_prob: dict = {}
    for state in forward_prob:
        state_total_prob[state] = forward_prob[state] * backward_prob[state]
    return state_total_prob

In [129]:
def get_total_prob(state_total_prob: dict, observations: list) -> torch.DoubleTensor:
    combined_state_totals = [state_total_prob[state] for state in state_total_prob]
    return torch.stack(combined_state_totals, dim=0).sum(dim=0)

In [130]:
def get_new_state_prob(state_total_prob: dict, total_prob: torch.DoubleTensor) -> dict:
    new_state_prob: dict = {}
    for state in state_total_prob:
        new_state_prob[state] = state_total_prob[state] / total_prob
    return new_state_prob

In [131]:
def get_observation_state_prob(new_state_prob: dict, observations: list) -> dict:
    observation_state_prob: dict = {}
    for state in new_state_prob:
        observation_state_prob[state] = {}
        for tag in set(observations):
            observation_state_prob[state][tag] = torch.DoubleTensor([])
            for idx, observation in enumerate(observations):
                probability = new_state_prob[state][idx] if observation == tag else 0
                observation_state_prob[state][tag] = torch.cat((observation_state_prob[state][tag], torch.DoubleTensor([probability])))
    return observation_state_prob

In [132]:
def get_transition_state_prob(forward_prob: dict, backward_prob: dict, total_prob: torch.DoubleTensor, observations: list) -> dict:
    transition_state_prob: dict = {}
    for prev_state in emission:
        transition_state_prob[prev_state] = {}
        for state in emission:
            transition_state_prob[prev_state][state] = torch.DoubleTensor([])
            for i, observation in enumerate(observations[1:]):
                idx = i + 1
                probability = forward_prob[prev_state][idx-1]*backward_prob[state][idx]
                probability *= transition[prev_state][state]*emission[state][observation]
                probability /= total_prob[idx]
                transition_state_prob[prev_state][state] = torch.cat((transition_state_prob[prev_state][state], torch.DoubleTensor([probability])))
    return transition_state_prob

In [133]:
def get_emissions(observation_state_prob: dict, new_state_prob: dict) -> dict:
    emissions = {}
    for state in new_state_prob:
        emissions[state] = {}
        for tag in observation_state_prob[state]:
            emissions[state][tag] = torch.sum(observation_state_prob[state][tag])/torch.sum(new_state_prob[state])
    return emissions

In [134]:
def get_transitions(transition_state_prob: dict, new_state_prob: dict) -> dict:
    transitions = {}
    for prev_state in transition_state_prob:
        transitions[prev_state] = {}
        for state in transition_state_prob:
            transitions[prev_state][state] = torch.sum(transition_state_prob[prev_state][state])/torch.sum(new_state_prob[state])
        transitions[prev_state]['STOP'] = new_state_prob[prev_state][-1]/torch.sum(new_state_prob[prev_state])
    return transitions

In [135]:
def get_init(new_state_prob: dict) -> dict:
    init = {}
    for state in new_state_prob:
        init[state] = new_state_prob[state][0]
    return init

In [136]:
def iterate(observations: list):
    forward: dict = get_forward_prob(observations)
    backward: dict = get_backward_prob(observations)
    state_total: dict = get_state_total_prob(forward, backward)
    total: torch.DoubleTensor = get_total_prob(state_total, observations)
    new_state: dict = get_new_state_prob(state_total, total)
    observation_state: dict = get_observation_state_prob(new_state, observations)
    transition_state: dict = get_transition_state_prob(forward, backward, total, observations)
    emissions = get_emissions(observation_state, new_state)
    transitions = get_transitions(transition_state, new_state)
    init = get_init(new_state)
    return init, transitions, emissions

In [137]:
sequences: list = []
sequence_labels: list = []
current: list = []
current_labels: list = []
sequence_size: int = 10
print("Dividing unsupervised data...")
for idx, character in enumerate(val_sequence):
    current.append(character)
    current_labels.append(val_labels[idx])
    if idx % sequence_size == 0 and idx > 0:
        sequences.append(current)
        sequence_labels.append(current_labels)
        current = []
        current_labels = []
print("{} sequences of unsupervised data of length {}".format(len(sequences), sequence_size))

Dividing unsupervised data...
1217 sequences of unsupervised data of length 10


In [138]:
def has_nans(d: dict) -> bool:
    for i in d.values():
        if isinstance(i,dict):
            if has_nans(i):
                return True
        else:
            if i != i:
                return True
    return False

In [139]:
print("Fitting...")
for sequence in sequences[:200000]:
    try:
        batch_initials, batch_transitions, batch_emissions = iterate(sequence)
        transition = batch_transitions if not has_nans(batch_transitions) else transition
        if not has_nans(batch_emissions):
            for state in emission:
                emission[state].update(batch_emissions)
    except:
        pass
print("Generated probabilities based on unlabeled data.")

Fitting...
Generated probabilities based on unlabeled data.


In [140]:
class Node:
    def __init__(self, state: str, probability: float, back_pointer):
        self.back_pointer: Node = back_pointer
        self.state: str = state
        self.probability: float = probability


def keys_match(dict_a: dict, dict_b: dict) -> bool:
    return dict_a.keys() == dict_b.keys()


def key_with_max_val(d: dict) -> str:
    """https://stackoverflow.com/questions/268272/getting-key-with-maximum-value-in-dictionary"""
    v = list(d.values())
    k = list(d.keys())
    return k[v.index(max(v))]


def node_with_max_prob(d: dict) -> Node:
    max_node = Node(None, 0.0, None)
    for node in d.values():
        if node.probability > max_node.probability:
            max_node = node
    return max_node


class Viterbi:
    def __init__(self, initial_probabilities: dict, transition_probabilities: dict, emission_probabilities: dict,):
        assert keys_match(initial_probabilities, emission_probabilities) and\
               keys_match(initial_probabilities, transition_probabilities), "Hidden states must be consistent!"
        self.initial = initial_probabilities
        self.emission = emission_probabilities
        self.transitions = transition_probabilities

    def predict_path(self, observations: list) -> list:
        matrix: list = [{}]

        for state in self.initial:
            matrix[0][state] = Node(state, self.initial[state]*self.emission[state][observations[0]], None)

        # fill initial probabilities
        for prev_idx, observation in enumerate(observations[1:]):
            matrix.append({})
            for state in self.transitions:
                transitions: dict = {}
                for prev_state in matrix[prev_idx]:
                    prev_prob = matrix[prev_idx][prev_state].probability
                    transition_prob = self.transitions[prev_state][state]*prev_prob
                    transitions[prev_state] = transition_prob
                last_state = key_with_max_val(transitions)
                probability = self.emission[state][observation]*transitions[last_state]
                matrix[prev_idx+1][state] = Node(state, probability, matrix[prev_idx][last_state])

        current_node: Node = node_with_max_prob(matrix[-1])
        sequence: list = []
        while current_node is not None:
            sequence.insert(0, current_node.state)
            current_node = current_node.back_pointer

        return sequence

In [141]:
viterbi = Viterbi(initial, transition, emission)

In [142]:
def get_accuracy(preds: list, labels: list) -> float:
    corrects: list = []
    for idx, pred in enumerate(preds):
        corrects.append(pred == labels[idx])
    return sum(corrects) / len(corrects)

In [143]:
def get_precision(preds, y):
    true_positive_preds = 0
    positive_preds = 0
    for idx, pred in enumerate(preds):
        if pred == '1':
            positive_preds += 1
            if y[idx] == '1':
                true_positive_preds += 1
    return true_positive_preds / positive_preds

In [144]:
def get_recall(preds, y):
    true_positive_preds = 0
    positive_labels = 0
    for idx, label in enumerate(y):
        if label == '1':
            positive_labels += 1
            if preds[idx] == '1':
                true_positive_preds += 1
    return true_positive_preds / positive_labels

In [145]:
def f1score(precision, recall):
    return 2*((precision*recall)/(precision+recall))

In [146]:
acc = 0
precision = 0
recall = 0
idx = 0
print("Evaluating...")
for sequence in sequences:
    try:
        predictions = viterbi.predict_path(sequence)
        labels = sequence_labels[idx]
        a = get_accuracy(predictions, labels)
        p = get_precision(predictions, labels)
        r = get_recall(predictions, labels)
        # make sure all functions succeed, then accumulate
        acc += a
        precision += p
        recall += r
        idx += 1
    except:
        pass
precision = precision / (idx+1)
recall = recall / (idx+1)
print("Validation Metrics:")
print("\tAccuracy: ", "{:0.2f}%".format(acc / (idx+1) * 100))
print("\tPrecision: ", "{:0.2f}".format(precision))
print("\tRecall: ", "{:0.2f}".format(recall))
print("\tF1 Score: ", "{:0.2f}".format(f1score(precision, recall)))

Evaluating...
Validation Metrics:
	Accuracy:  0.00%
	Precision:  0.00
	Recall:  0.00


ZeroDivisionError: ignored

In [ ]:
if "test_data" in globals():
    test_sequence = [pair[0] for pair in test_data]
    test_labels = [pair[1] for pair in test_data]
    sequences: list = []
    sequence_labels: list = []
    current: list = []
    current_labels: list = []
    sequence_size: int = 10
    print("Dividing test data...")
    for idx, character in enumerate(test_sequence):
        current.append(character)
        current_labels.append(test_labels[idx])
        if idx % sequence_size == 0 and idx > 0:
            sequences.append(current)
            sequence_labels.append(current_labels)
            current = []
            current_labels = []
    print("{} sequences of test data of length {}".format(len(sequences), sequence_size))
    
    acc = 0
    precision = 0
    recall = 0
    idx = 0
    print("Evaluating...")
    for sequence in sequences:
        try:
            predictions = viterbi.predict_path(sequence)
            labels = sequence_labels[idx]
            a = get_accuracy(predictions, labels)
            p = get_precision(predictions, labels)
            r = get_recall(predictions, labels)
            # make sure all functions succeed, then accumulate
            acc += a
            precision += p
            recall += r
            idx += 1
        except:
            pass
    precision = precision / (idx+1)
    recall = recall / (idx+1)
    print("Testing Metrics:")
    print("\tAccuracy: ", "{:0.2f}%".format(acc / (idx+1) * 100))
    print("\tPrecision: ", "{:0.2f}".format(precision))
    print("\tRecall: ", "{:0.2f}".format(recall))
    print("\tF1 Score: ", "{:0.2f}".format(f1score(precision, recall)))